### Endoderm perturb-seq

This notebook is for exploratory analysis and reformatting of the Genga et al. [endoderm directed differentiation perturb-seq data](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6525305/).

Always run `maehr.ipynb` before this notebook. This notebook relies on its outputs. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import regex as re
import os
import shutil
import importlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from scipy.stats import spearmanr as spearmanr
from scipy.sparse import csr_matrix
from IPython.display import display, HTML
from collections import Counter

# local
import pereggrn_perturbations
import importlib
import sys
sys.path.append("setup")
import ingestion
import global_effects
importlib.reload(ingestion)
importlib.reload(global_effects)

import anndata
import os, sys
import itertools as it
from scipy.stats import spearmanr, pearsonr, rankdata, f_oneway
from statsmodels.stats.multitest import multipletests
from sklearn.metrics import mutual_info_score
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

#      visualization settings
%matplotlib inline
plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

# I prefer to specify the working directory explicitly.
os.chdir("/home/ekernf01/Desktop/jhu/research/projects/perturbation_prediction/cell_type_knowledge_transfer/perturbation_data")

pereggrn_perturbations.set_data_path('perturbations')

# Universal
geneAnnotationPath = "../accessory_data/gencode.v35.annotation.gtf.gz"       # Downloaded from https://www.gencodegenes.org/human/release_35.html
humanTFPath =  "../accessory_data/humanTFs.csv"                              # Downloaded from http://humantfs.ccbr.utoronto.ca/download.php
humanEpiPath = "../accessory_data/epiList.csv"                               # Downloaded from https://epifactors.autosome.org/description 
cellcycleGenePath = "../accessory_data/regev_lab_cell_cycle_genes.txt"

# dataset-specific
dataset_name = "definitive_endoderm"

We start from the raw counts. Here is what the original authors write about filtering. 

> In each replicate, genes appearing in only one cell were excluded. Cells with doublet-modeling log10 LR above 0.2 were excluded (730 cells). Cells were excluded if no gRNAs were detected (29 cells). Cells assigned to scramble gRNA #5 were excluded based on the high number of differentially expressed transcripts (88 cells). This left 548 negative control cells (3.4%), which is lower than expected but still within the range used by similar studies (Dixit et al., 2016). Ten gRNAs yielded exactly 0 counts in the END gDNA ScreenProcessing results, and upon visualization of the RNA alignments, these were found to contain possible mutations or targeting errors (they were: ARNT_gRNA2, ATF3_gRNA3, CREB3_gRNA2, FOXA2_gRNA3, FOXA3_gRNA1, GATA4_gRNA1, GATA6_gRNA2, JUND_gRNA2, TGIF2_gRNA1, ZNF263_gRNA2). Cells assigned to these gRNAs were excluded from downstream analysis (634 cells). After all exclusions, 16,110 cells remained.

We assign each cell to the most abundant gRNA. (Note that this is an arrayed screen, not a pooled screen; **no** cell will have more than one gRNA.) Here, we exclude: 

- cells assigned to scramble number 5 or to guides with possible mutations or targeting errors 
- We exclude any cell with (strictly) fewer than 3 UMIs supporting the most abundant gRNA
- cells in which EOMES, MYBL2, and CDC5L were KD'd. There are too few cells observed (<15 cells each). This is an important finding, but not useful for understanding the *transcriptomic* response to perturbation. 
- cells in which OTX2, PITX2, or NR5A2 were KD'd, because these genes did not always decrease in expression. 

In [ ]:
expression_quantified = anndata.concat({
    "rep1":sc.read_10x_mtx("not_ready/genga_definitive_endoderm/rep1_filtered"),
    "rep2":sc.read_10x_mtx("not_ready/genga_definitive_endoderm/rep2_filtered"),
}, index_unique="-")
print(expression_quantified.shape)
gRNA_counts = anndata.concat({
    "rep1":sc.read_10x_mtx("not_ready/genga_definitive_endoderm/grna amplified libraries/rep1"),
    "rep2":sc.read_10x_mtx("not_ready/genga_definitive_endoderm/grna amplified libraries/rep2"),
}, index_unique="-")

In [ ]:
# Guide assignment
gRNA_counts = gRNA_counts[expression_quantified.obs_names, :]
gRNA_indices = expression_quantified.var_names[expression_quantified.var_names.str.contains("gRNA")]
gRNA_indices = np.array([g for g in gRNA_indices])
gRNA_counts = expression_quantified[:, gRNA_indices]
expression_quantified.obs["total_gRNA_counts"] = pd.Series(
    np.array(gRNA_counts.X.sum(axis=1)).squeeze()
)
expression_quantified.obs["most_abundant_gRNA"] = [",".join(gRNA_indices[i][0]) for i in gRNA_counts.X.argmax(1)]
expression_quantified.obs["total_gRNA_counts"] = gRNA_counts.X.sum(1)
expression_quantified.obs["most_abundant_gRNA_counts"] = gRNA_counts.X.toarray().max(1)
print(expression_quantified.obs["most_abundant_gRNA"].value_counts().to_csv())

# Guide UMI counts -- most abundant vs total
plt.hist2d(
    expression_quantified.obs["total_gRNA_counts"],
    expression_quantified.obs["most_abundant_gRNA_counts"], 
    bins = (200,200),
)
plt.xlim(0,40)
plt.ylim(0,40)
plt.xlabel("total_gRNA_counts")
plt.ylabel("most_abundant_gRNA_counts")

# Required standard metadata
expression_quantified.obs["perturbation"]   = [s.split("_")[0] for s in expression_quantified.obs["most_abundant_gRNA"]]
print(expression_quantified.obs["perturbation"].value_counts())
expression_quantified.obs["is_control"]     = expression_quantified.obs["perturbation"].isin(["Scramble"])
print(expression_quantified.obs["is_control"].value_counts())
expression_quantified.obs["is_control_int"] = [1 if b else 0 for b in expression_quantified.obs["is_control"]]
expression_quantified.obs["timepoint"]      = 4

# EXCLUSIONS
gRNAs_to_exclude = ["ARNT_gRNA2", "ATF3_gRNA3", "CREB3_gRNA2", "FOXA2_gRNA3", "FOXA3_gRNA1", "GATA4_gRNA1", "GATA6_gRNA2", "JUND_gRNA2", "TGIF2_gRNA1", "ZNF263_gRNA2", "Scramble_gRNA5"]
perts_to_exclude = ["EOMES", "CDC5L", "MYBL2", "NR5A2", "OTX2", "PITX2"]
expression_quantified = expression_quantified[~expression_quantified.obs["most_abundant_gRNA"].isin(gRNAs_to_exclude), :]
expression_quantified = expression_quantified[~expression_quantified.obs["perturbation"].isin(perts_to_exclude), :]
expression_quantified = expression_quantified[expression_quantified.obs["most_abundant_gRNA_counts"] >= 3, :]

### Cell cycle annotation

Following the Figure 1 and S1 notebook from [Petrus-Reurer et al.](https://github.com/lamanno-epfl/rpe_differentiation_profiling_code/blob/main/JupyterNotebooks/HTML/Figure1_S1.html). 


In [ ]:
S_genes_hum = ["MCM5", "PCNA", "TYMS", "FEN1", "MCM2", "MCM4", "RRM1", "UNG", "GINS2", 
            "MCM6", "CDCA7", "DTL", "PRIM1", "UHRF1", "CENPU", "HELLS", "RFC2", 
            "RPA2", "NASP", "RAD51AP1", "GMNN", "WDR76", "SLBP", "CCNE2", "UBR7", 
            "POLD3", "MSH2", "ATAD2", "RAD51", "RRM2", "CDC45", "CDC6", "EXO1", "TIPIN", 
            "DSCC1", "BLM", "CASP8AP2", "USP1", "CLSPN", "POLA1", "CHAF1B", "BRIP1", "E2F8"]
G2M_genes_hum = ["HMGB2", "CDK1", "NUSAP1", "UBE2C", "BIRC5", "TPX2", "TOP2A", "NDC80",
             "CKS2", "NUF2", "CKS1B", "MKI67", "TMPO", "CENPF", "TACC3", "PIMREG", 
             "SMC4", "CCNB2", "CKAP2L", "CKAP2", "AURKB", "BUB1", "KIF11", "ANP32E", 
             "TUBB4B", "GTSE1", "KIF20B", "HJURP", "CDCA3", "JPT1", "CDC20", "TTK",
             "CDC25C", "KIF2C", "RANGAP1", "NCAPD2", "DLGAP5", "CDCA2", "CDCA8", "ECT2", 
             "KIF23", "HMMR", "AURKA", "PSRC1", "ANLN", "LBR", "CKAP5", "CENPE", 
             "CTCF", "NEK2", "G2E3", "GAS2L3", "CBX5", "CENPA"]
sc.tl.score_genes_cell_cycle(expression_quantified, s_genes=S_genes_hum, g2m_genes=G2M_genes_hum)

### Normalization and variable gene ranking

We select the same genes and rank them in the same way as the Maehr group time-series data. We snag the timeseries data and remove the genes that aren't present in the Genga data.

We also run basic total count normalization, and we log1p-transform the data.

In [ ]:
time_series = pereggrn_perturbations.load_perturbation("definitive_endoderm", is_timeseries=True)
pd.Series(time_series.var.index.difference(expression_quantified.var.index)).to_csv("perturbations/definitive_endoderm/genes_missing_from_genga.csv")
pd.Series(expression_quantified.var.index.difference(time_series.var.index)).to_csv("perturbations/definitive_endoderm/genes_missing_from_cuomo.csv")
pd.Series(expression_quantified.var.index.intersection(time_series.var.index)).to_csv("perturbations/definitive_endoderm/genes_shared.csv")
shared_genes = expression_quantified.var.index.intersection(time_series.var.index)
time_series           = time_series[:,shared_genes]
expression_quantified = expression_quantified[:,shared_genes]
expression_quantified.uns["perturbed_and_measured_genes"]     = list(set(expression_quantified.obs["perturbation"]).difference({"Scramble"}).intersection(expression_quantified.var_names))
expression_quantified.uns["perturbed_but_not_measured_genes"] = list(set(expression_quantified.obs["perturbation"]).difference({"Scramble"}).difference(expression_quantified.var_names))
print(expression_quantified.uns["perturbed_and_measured_genes"])
print(expression_quantified.uns["perturbed_but_not_measured_genes"])

expression_quantified.raw = expression_quantified.copy()
sc.pp.normalize_total(expression_quantified)
sc.pp.log1p(expression_quantified)


### EDA and cell type annotation

(Original cell type labels are not published.)

In [ ]:
sc.pp.highly_variable_genes(expression_quantified, n_bins=50, n_top_genes = 500, flavor = "seurat_v3" )
sc.pp.scale(expression_quantified)
with warnings.catch_warnings():
    sc.tl.pca(expression_quantified, n_comps=10)
sc.pp.neighbors(expression_quantified)
sc.tl.umap(expression_quantified)

In [ ]:
sc.tl.louvain(expression_quantified, resolution = 0.124)
plt.rcParams['figure.figsize'] = [6, 4.5]
labels = {'0':"endoderm", '1':"pluripotent", '2':"ectopic", '3': "mesendoderm"}
expression_quantified.obs["cell_type"] = [labels[l] for l in expression_quantified.obs["louvain"]]
stuff_to_show = [ 
       "phase", "louvain", "cell_type",
       "SOX17", "FOXA2", "EOMES", "NANOG", "SOX2", "POU5F1", "T", "MIXL1"]
sc.pl.umap(expression_quantified, color = stuff_to_show)
sc.pl.pca(expression_quantified, color = stuff_to_show)

### Effects on differentiation

There are probably some errors in cluster assignment and gRNA assignment. After looking at the joint distribution, we will keep only combinations with at least 40 cells. 

In [ ]:
cell_type_and_pert = expression_quantified.obs[["cell_type", "perturbation"]].value_counts().reset_index()
print(cell_type_and_pert.pivot(index = "perturbation", columns = "cell_type", values = "count").fillna(0))
cell_type_and_pert["keep"] = cell_type_and_pert["count"] >= 40
expression_quantified.obs = expression_quantified.obs.merge(cell_type_and_pert[["cell_type", "perturbation", "keep"]], on = ["cell_type", "perturbation"], how = "left")
print(expression_quantified.obs["keep"].value_counts())
expression_quantified = expression_quantified[expression_quantified.obs["keep"], :]

### Undo the unwanted parts of the EDA

In [ ]:
expression_quantified.var["highly_variable_rank"] = time_series.var["highly_variable_rank"]
expression_quantified.X = expression_quantified.raw.X.copy()
sc.pp.normalize_total(expression_quantified)
sc.pp.log1p(expression_quantified)
print(expression_quantified.X.min())
expression_quantified.X.max()

### Perturbation effects

Check if the KD'd gene went down and then check how much the rest of the transcriptome changed. 

In [ ]:
expression_quantified = ingestion.describe_perturbation_effect(expression_quantified, "knockdown")

### Save the data

In [ ]:
pereggrn_perturbations.check_perturbation_dataset(ad = expression_quantified, is_timeseries=False, is_perturbation=True)
pereggrn_perturbations.check_perturbation_dataset(ad = time_series, is_timeseries=True, is_perturbation=False)
time_series.write_h5ad(os.path.join(f"perturbations/{dataset_name}", f"train.h5ad"))
expression_quantified.write_h5ad(os.path.join(f"perturbations/{dataset_name}", f"test.h5ad"))